In [ ]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import Database
import pandas as pd
import numpy as np
import sys
import Database
import Algorithm
import json



In [3]:
try:
    collection = Database.Connect()
    #all_data = collection.find().sort("name",1)
except Exception as e:
    print("Kết nối thất bại!")
    sys.exit(1)

# adj_list, vertices, edges = Database.Load_Graph(collection,'674c805de3251455aa5d1c95')
# #print(adj_list)

# edges_list = Database.adj_list_to_edges_list(adj_list,vertices)
# for v_f,v_t,w in edges_list:
#     print(f"{v_f} {v_t} {w}")

In [3]:

# Kết nối đến Neo4j Aura
NEO4J_URI = "neo4j+s://45b7324c.databases.neo4j.io"  # Thay bằng URI từ Neo4j Aura
USERNAME = "neo4j"
PASSWORD = "5l5KqmekdQxJ9vSz5sR8dQeGciU-rFBk61pt8b47IYk"  # Thay bằng mật khẩu của bạn

driver = GraphDatabase.driver(NEO4J_URI, auth=(USERNAME, PASSWORD))

In [8]:
def delete_all_data():
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")
        print("Đã xóa toàn bộ dữ liệu.")

# Thực thi
delete_all_data()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_17572\2203686074.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


Đã xóa toàn bộ dữ liệu.


In [9]:


def upload_to_neo4j(json_file):
    # Đọc file JSON
    with open(json_file, "r", encoding="utf-8") as file:
        data = json.load(file)

    # Kết nối với Neo4j
    with driver.session() as session:
        for graph_data in data:
            graph_id = graph_data["_id"]
            graph_name = graph_data["name"]  # Tên đồ thị
            directed = 'directed' in graph_data.get('tags', [])
            edges = graph_data["edges"]

            # Tạo node cho đồ thị
            session.run("""
                MERGE (g:Graph {id: $graph_id, name: $graph_name})
            """, graph_id=graph_id, graph_name=graph_name)

            # Tạo các đỉnh và cạnh trong Neo4j
            for edge in edges:
                v_from = edge["v_from"]
                v_to = edge["v_to"]
                weight = edge["weight"]

                # Tạo các đỉnh với thuộc tính `graph_id` và liên kết với đồ thị
                session.run("""
                    MERGE (a:Node {name: $v_from, graph_id: $graph_id})
                    MERGE (b:Node {name: $v_to, graph_id: $graph_id})
                    MERGE (g:Graph {id: $graph_id})
                    MERGE (g)-[:CONTAINS]->(a)
                    MERGE (g)-[:CONTAINS]->(b)
                """, v_from=v_from, v_to=v_to, graph_id=graph_id)

                # Tạo các cạnh giữa các đỉnh
                if directed:
                    session.run("""
                        MERGE (a:Node {name: $v_from, graph_id: $graph_id})
                        MERGE (b:Node {name: $v_to, graph_id: $graph_id})
                        MERGE (a)-[:CONNECTED {weight: $weight}]->(b)
                    """, v_from=v_from, v_to=v_to, graph_id=graph_id, weight=weight)
                else:
                    session.run("""
                        MERGE (a:Node {name: $v_from, graph_id: $graph_id})
                        MERGE (b:Node {name: $v_to, graph_id: $graph_id})
                        MERGE (a)-[:CONNECTED {weight: $weight}]->(b)
                        MERGE (b)-[:CONNECTED {weight: $weight}]->(a)
                    """, v_from=v_from, v_to=v_to, graph_id=graph_id, weight=weight)

            print(f"Đã tải đồ thị với _id: {graph_id} ({graph_name}) lên Neo4j.")


# Thực thi
json_file = "database.json"
upload_to_neo4j(json_file)

# Đóng kết nối
driver.close()


C:\Users\ASUS\AppData\Local\Temp\ipykernel_17572\1949779598.py:7: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


Đã tải đồ thị với _id: 673616adf36b60bb91cb64a0 (Mohammed’s scimitars) lên Neo4j.
Đã tải đồ thị với _id: 6736178bf36b60bb91cb64a2 (Möbius Graph) lên Neo4j.
Đã tải đồ thị với _id: 67361a0ef36b60bb91cb64a7 (Clebsch Graph) lên Neo4j.
Đã tải đồ thị với _id: 674823178732f13e3bace2c4 (Graph01) lên Neo4j.
Đã tải đồ thị với _id: 674c74cee3251455aa5d1c90 (Graph02) lên Neo4j.
Đã tải đồ thị với _id: 674c775ce3251455aa5d1c91 (Graph03) lên Neo4j.
Đã tải đồ thị với _id: 674c7dfae3251455aa5d1c92 (Graph04) lên Neo4j.
Đã tải đồ thị với _id: 674c7eebe3251455aa5d1c93 (Graph05) lên Neo4j.
Đã tải đồ thị với _id: 674c7f97e3251455aa5d1c94 (Graph06) lên Neo4j.
Đã tải đồ thị với _id: 674c805de3251455aa5d1c95 (Graph07) lên Neo4j.
Đã tải đồ thị với _id: 674c80f1e3251455aa5d1c96 (Graph08) lên Neo4j.
Đã tải đồ thị với _id: 674c8236e3251455aa5d1c97 (Graph09) lên Neo4j.
Đã tải đồ thị với _id: 674c82d1e3251455aa5d1c98 (Graph10) lên Neo4j.
Đã tải đồ thị với _id: 674c8407e3251455aa5d1c99 (Graph11) lên Neo4j.
Đã tải đồ 

In [10]:
def fetch_graph_data():
    with driver.session() as session:
        # Truy vấn Cypher
        query = """
        MATCH (g:Graph)-[:CONTAINS]->(n:Node)
        RETURN g.id AS graph_id, g.name AS graph_name, COUNT(n) AS node_count
        """
        results = session.run(query)
        
        # In dữ liệu
        for record in results:
            print(f"ID: {record['graph_id']}, Tên: {record['graph_name']}, Số đỉnh: {record['node_count']}")

# Thực thi
fetch_graph_data()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_17572\3072559520.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


ID: 673616adf36b60bb91cb64a0, Tên: Mohammed’s scimitars, Số đỉnh: 11
ID: 6736178bf36b60bb91cb64a2, Tên: Möbius Graph, Số đỉnh: 6
ID: 67361a0ef36b60bb91cb64a7, Tên: Clebsch Graph, Số đỉnh: 16
ID: 674823178732f13e3bace2c4, Tên: Graph01, Số đỉnh: 5
ID: 674c74cee3251455aa5d1c90, Tên: Graph02, Số đỉnh: 50
ID: 674c775ce3251455aa5d1c91, Tên: Graph03, Số đỉnh: 6
ID: 674c7dfae3251455aa5d1c92, Tên: Graph04, Số đỉnh: 17
ID: 674c7eebe3251455aa5d1c93, Tên: Graph05, Số đỉnh: 4
ID: 674c7f97e3251455aa5d1c94, Tên: Graph06, Số đỉnh: 6
ID: 674c805de3251455aa5d1c95, Tên: Graph07, Số đỉnh: 8
ID: 674c80f1e3251455aa5d1c96, Tên: Graph08, Số đỉnh: 5
ID: 674c8236e3251455aa5d1c97, Tên: Graph09, Số đỉnh: 6
ID: 674c82d1e3251455aa5d1c98, Tên: Graph10, Số đỉnh: 9
ID: 674c8407e3251455aa5d1c99, Tên: Graph11, Số đỉnh: 11
ID: 674c84c7e3251455aa5d1c9a, Tên: Graph12, Số đỉnh: 6
ID: 674c8632e0e23c92abe7fc06, Tên: Graph33, Số đỉnh: 8
ID: 674c88bae0e23c92abe7fc07, Tên: Graph34, Số đỉnh: 12
ID: 674c8a3ee3251455aa5d1c9b, Tên: 

In [ ]:
def get_adjacency_list(graph_id):
    with driver.session() as session:
        # Truy vấn các cạnh của đồ thị
        query = """
        MATCH (g:Graph {id: $graph_id})-[:CONTAINS]->(n1:Node)
        MATCH (n1)-[r:CONNECTED]->(n2:Node)
        RETURN n1.name AS from_node, n2.name AS to_node, r.weight AS weight
        """
        results = session.run(query, graph_id=graph_id)
        
        # Xây dựng danh sách kề
        adj_list = {}
        for record in results:
            from_node = record["from_node"]
            to_node = record["to_node"]
            weight = record["weight"]
            
            # Thêm vào danh sách kề
            if from_node not in adjacency_list:
                adj_list[from_node] = []
            adj_list[from_node].append((to_node, weight))
        
        return adjacency_list

# Thực thi
adjacency_list = get_adjacency_list("674df8fbb2c1143f2130cf15")
print("Danh sách kề có trọng số:")
print(adjacency_list)

Danh sách kề có trọng số:
{'v01': [('v02', 1), ('v05', 1), ('v06', 1)], 'v02': [('v01', 1), ('v06', 1), ('v03', 1), ('v07', 1)], 'v05': [('v01', 1), ('v06', 1)], 'v06': [('v01', 1), ('v02', 1), ('v05', 1)], 'v03': [('v02', 1), ('v07', 1), ('v04', 1)], 'v07': [('v02', 1), ('v03', 1), ('v04', 1)], 'v04': [('v03', 1), ('v07', 1), ('v08', 1)], 'v08': [('v04', 1)], 'v09': [('v10', 1), ('v11', 1), ('v12', 1)], 'v10': [('v09', 1)], 'v11': [('v09', 1)], 'v12': [('v09', 1), ('v13', 1), ('v14', 1)], 'v13': [('v12', 1), ('v14', 1)], 'v14': [('v12', 1), ('v13', 1)]}


C:\Users\ASUS\AppData\Local\Temp\ipykernel_17572\1377589945.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
